In [1]:
"""
This scripts merges the statistics of 2 sensors into 1
It does this for multiple sensors because i had added almost the same sensors via 2 different integrations
"""

'\nThis scripts merges the statistics of 2 sensors into 1\nIt does this for multiple sensors because i had added almost the same sensors via 2 different integrations\n'

In [2]:
hass_ip="192.168.1.25" # change this to the ip of you home assistant installation

In [8]:
# mapping of the sensors, statistics of the left ones will be merged into the right ones
mapping={'sensor.nibe_93714_44270': 'sensor.calc_cooling_supply_s1_44270',
        'sensor.nibe_93714_40013': 'sensor.bt7_hw_top_40013',
        'sensor.nibe_93714_40015': 'sensor.eb100_ep14_bt10_brine_in_temp_40015',
        'sensor.nibe_93714_40004': 'sensor.bt1_outdoor_temperature_40004',
        'sensor.nibe_93714_40012': 'sensor.eb100_ep14_bt3_return_temp_40012',
        'sensor.nibe_93714_40018': 'sensor.eb100_ep14_bt14_hot_gas_temp_40018',
        'sensor.nibe_93714_40017': 'sensor.eb100_ep14_bt12_condensor_out_40017',
        'sensor.nibe_93714_40033': 'sensor.bt50_room_temp_s1_40033',
        'sensor.nibe_93714_40067': 'sensor.bt1_average_40067',
        'sensor.nibe_93714_40016': 'sensor.eb100_ep14_bt11_brine_out_temp_40016',
        'sensor.nibe_93714_40014': 'sensor.bt6_hw_load_40014',
        'sensor.nibe_93714_43009': 'sensor.calc_supply_s1_43009',
        'sensor.nibe_93714_40019': 'sensor.eb100_ep14_bt15_liquid_line_40019',
        'sensor.nibe_93714_40008': 'sensor.bt2_supply_temp_s1_40008',
        'sensor.nibe_93714_40022': 'sensor.eb100_ep14_bt17_suction_40022'}

# Retrieve database from home assistant server

In [3]:
# add your public key to the homeassistant account to make scp possible
! scp homeassistant@{hass_ip}:/home/homeassistant/.homeassistant/home-assistant_v2.db .
! mkdir -p backup
! cp home-assistant_v2.db backup # make a backup

home-assistant_v2.db                          100%  762MB  11.3MB/s   01:07    


# Connect to DB

In [4]:
import pandas
import sqlite3
import pandas
pandas.options.plotting.backend = "plotly"

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("home-assistant_v2.db")

# Merge

In [9]:
def merge(source_sensor, target_sensor, table='statistics'):
    print(f"source: {source_sensor}, target: {target_sensor}")
    # read data from target sensor
    target_sensor_id=pandas.read_sql_query(f"""select id FROM statistics_meta 
                                                where statistic_id like '{target_sensor}';""", con).loc[0,'id']
    target_df=pandas.read_sql_query(
            f"select * FROM {table} where metadata_id = '{target_sensor_id}';", con
        )
    print(f"length of existing statistics for target sensor: {len(target_df)}")

    min_dt = target_df["start"].min()

    # read data from source sensor
    source_sensor_id=pandas.read_sql_query(f"""select id FROM statistics_meta 
                                                where statistic_id like '{source_sensor}';""", con).loc[0,'id']
    source_df=pandas.read_sql_query(
            f"select * FROM {table} where metadata_id = '{source_sensor_id}' and start < '{min_dt}';", con
        )
    print(f"length of existing statistics for source sensor: {len(source_df)}")


    # concat new historical statistics with updated old statistics
    df = pandas.concat([source_df, target_df], ignore_index=True).sort_values("start")

    # set metadata_id to the one of the target sensor
    df["metadata_id"]=target_sensor_id

    # drop all existing rows with same metadata_id as the sensors
    stmnt = f"""DELETE FROM {table}
                WHERE metadata_id in ({source_sensor_id}, {target_sensor_id});"""
    cur = con.cursor()
    cur.execute(stmnt)
    con.commit()

    # insert new data into table
    df.to_sql(
        table, con, schema=None, if_exists="append", index=False,
    )


In [10]:
for source_sensor, target_sensor in mapping.items():
    for table in ("statistics", "statistics_short_term"):    
        print(table)
        merge(source_sensor, target_sensor, table)

statistics
source: sensor.nibe_93714_44270, target: sensor.calc_cooling_supply_s1_44270
length of existing statistics for target sensor: 5102
length of existing statistics for source sensor: 0
statistics_short_term
source: sensor.nibe_93714_44270, target: sensor.calc_cooling_supply_s1_44270
length of existing statistics for target sensor: 2902
length of existing statistics for source sensor: 4564
statistics
source: sensor.nibe_93714_40013, target: sensor.bt7_hw_top_40013
length of existing statistics for target sensor: 5102
length of existing statistics for source sensor: 0
statistics_short_term
source: sensor.nibe_93714_40013, target: sensor.bt7_hw_top_40013
length of existing statistics for target sensor: 2902
length of existing statistics for source sensor: 4564
statistics
source: sensor.nibe_93714_40015, target: sensor.eb100_ep14_bt10_brine_in_temp_40015
length of existing statistics for target sensor: 5102
length of existing statistics for source sensor: 0
statistics_short_term
so

# Copy db back to home assistant server

In [11]:
! scp home-assistant_v2.db homeassistant@{hass_ip}:/home/homeassistant/.homeassistant/

home-assistant_v2.db                          100%  762MB  12.6MB/s   01:00    
